In [2]:
import dask.dataframe as dd
import pandas as pd

In [2]:
# Read the data using Dask
df_user = pd.read_csv('/home/nkama/masters_thesis_project/thesis/data/event_rec_engine_challenge/users.csv')
                    

In [3]:
df_user.head()

,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
2,823183725,en_US,1975,male,2012-10-06T03:14:07.149Z,Stratford Ontario,-240.0
3,1872223848,en_US,1991,female,2012-11-04T08:59:43.783Z,Tehran Iran,210.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0


In [4]:
"""
Load the saved user_event_pairs dataframe so we don't have to rerun all the code needed
to create the dataframe should the kernel crash when processing large data volume.
"""

user_event_pairs = pd.read_csv("/home/nkama/masters_thesis_project/thesis/interactions.csv")

user_event_pairs.head()

,Unnamed: 0,event_id,user_id,attendance_status
0,0,1159822043,1975964455,yes
1,1,1159822043,252302513,yes
2,2,1159822043,4226086795,yes
3,3,1159822043,3805886383,yes
4,4,1159822043,1420484491,yes


In [14]:
unique_users = user_event_pairs['user_id'].unique()

In [15]:
# Convert both to string type to ensure matching
user_event_pairs['user_id'] = user_event_pairs['user_id'].astype(str)
df_user['user_id'] = df_user['user_id'].astype(str)


In [16]:
# Filter users data
filtered_user_data = df_user[df_user["user_id"].isin(unique_users)]
filtered_user_data.head()

,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0
7,3473687777,id_ID,1965,female,2012-10-03T12:19:29.975Z,Medan Indonesia,420.0
12,1355996271,id_ID,1993,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,420.0


In [25]:
filtered_user_data.isnull().sum()

user_id      0
birthyear    0
gender       0
joinedAt     0
location     0
dtype: int64

In [ ]:
filtered_user_data = filtered_user_data.drop(["timezone","locale"],axis=1)

In [23]:
filtered_user_data = filtered_user_data.dropna()
filtered_user_data

,user_id,birthyear,gender,joinedAt,location
0,3197468391,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia
1,3537982273,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia
7,3473687777,1965,female,2012-10-03T12:19:29.975Z,Medan Indonesia
12,1355996271,1993,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia
13,2411726276,1996,female,2012-10-30T05:16:27.136Z,Triolet Mauritius
...,...,...,...,...,...
38187,3566369790,1989,male,2012-10-30T08:56:19.611Z,Jogjakarta Indonesia
38190,2630548270,1988,male,2012-10-29T16:02:19.692Z,Yogyakarta
38199,280076614,1995,female,2012-11-22T12:06:08.668Z,Phnom Penh
38203,3890944219,1993,female,2012-11-04T07:18:39.354Z,Kebumen Jawa Tengah Indonesia


In [24]:
len(filtered_user_data)

9922

In [26]:
# Print summary information
print(f"Original users: {len(df_user)}, Filtered users: {len(filtered_user_data)}")

Original users: 38209, Filtered users: 9922


Extract age column from the birtday and event start time

In [27]:
events_data = pd.read_csv("/home/nkama/masters_thesis_project/thesis/data/event_rec_engine_challenge/events.csv",\
                           usecols=range(3))
#events_data['start_time'] = pd.to_datetime(events_data['start_time'], errors='coerce')
events_data = events_data.dropna(subset=['start_time'])

In [31]:
from datetime import datetime


# Create a copy of the filtered data to avoid the warning
filtered_user_data = filtered_user_data.copy()

# Convert birthyear to int and calculate age
filtered_user_data.loc[:, 'birthyear'] = filtered_user_data['birthyear'].astype(int)
# First, convert string timestamps to datetime objects
events_data['start_time'] = pd.to_datetime(events_data['start_time'],errors='coerce')
#filtered_user_data['joinedAt'] = pd.to_datetime(filtered_user_data['joinedAt'])

# Extract reference year - let's use the median event year as our reference point
# This is a balanced approach that represents the "typical" time in the dataset
median_event_date = events_data['start_time'].median()
reference_year = median_event_date.year
filtered_user_data.loc[:, 'age'] = reference_year - filtered_user_data['birthyear']

# Verify the changes
print("Sample of data with age:")
print(filtered_user_data[['user_id', 'birthyear', 'age']].head())

# Check for any invalid ages
invalid_ages = filtered_user_data[(filtered_user_data['age'] < 0) | (filtered_user_data['age'] > 100)]
print(f"\nNumber of invalid ages: {len(invalid_ages)}")



Sample of data with age:
       user_id birthyear age
0   3197468391      1993  19
1   3537982273      1992  20
7   3473687777      1965  47
12  1355996271      1993  19
13  2411726276      1996  16

Number of invalid ages: 30


In [29]:
len(filtered_user_data)

9922

In [30]:
filtered_user_data.isnull().sum()

user_id      0
birthyear    0
gender       0
joinedAt     0
location     0
age          0
dtype: int64

In [36]:
filtered_user_data = filtered_user_data.drop("birthyear",axis=1)


In [37]:
filtered_user_data.head()


,user_id,gender,joinedAt,location,age
0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19
1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20
7,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47
12,1355996271,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,19
13,2411726276,female,2012-10-30T05:16:27.136Z,Triolet Mauritius,16


In [48]:
filtered_user_data.reset_index(drop=True)

,user_id,gender,joinedAt,location,age
0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19
1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20
2,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47
3,1355996271,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,19
4,2411726276,female,2012-10-30T05:16:27.136Z,Triolet Mauritius,16
...,...,...,...,...,...
9917,3566369790,male,2012-10-30T08:56:19.611Z,Jogjakarta Indonesia,23
9918,2630548270,male,2012-10-29T16:02:19.692Z,Yogyakarta,24
9919,280076614,female,2012-11-22T12:06:08.668Z,Phnom Penh,17
9920,3890944219,female,2012-11-04T07:18:39.354Z,Kebumen Jawa Tengah Indonesia,19


In [113]:
filtered_user_data.to_csv("filtered_user_data.csv")

In [39]:
from get_user_location_data import add_user_coordinates
import pandas as pd


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import time


df = filtered_user_data.copy()
# Initialize geocoder
geolocator = Nominatim(user_agent="location_coordinates_finder")

# Function to get latitude and longitude
def get_coordinates(location):
    try:
        # Add small delay to respect geocoding service rate limits
        time.sleep(1)
        location_data = geolocator.geocode(location)
        if location_data:
            return location_data.latitude, location_data.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {location}: {e}")
        return None, None

# Add latitude and longitude columns
df[['lat', 'lon']] = df['location'].apply(lambda x: pd.Series(get_coordinates(x)))

# Display the result
#df

In [102]:
df.head()

,user_id,gender,joinedAt,location,age
0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19
1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20
7,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47
12,1355996271,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,19
13,2411726276,female,2012-10-30T05:16:27.136Z,Triolet Mauritius,16


In [106]:
# import the add_user_coordinates function to retrieve user cordinates
from get_user_location_data import add_user_coordinates

In [ ]:
users_df = add_user_coordinates(filtered_user_data[["location"]])

In [3]:
filtered_user_data = pd.read_csv("/home/nkama/masters_thesis_project/thesis/filtered_user_data.csv")
users_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/users_location.csv")

In [10]:
len(filtered_user_data), len(users_df)

(9922, 9922)

In [5]:
filtered_user_data.head()

,Unnamed: 0,user_id,gender,joinedAt,location,age
0,0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19
1,1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20
2,7,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47
3,12,1355996271,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,19
4,13,2411726276,female,2012-10-30T05:16:27.136Z,Triolet Mauritius,16


In [4]:
users_df.head()

,Unnamed: 0,location,lat,lng
0,0,Medan Indonesia,3.590,98.674
1,1,Medan Indonesia,3.590,98.674
2,7,Medan Indonesia,3.590,98.674
3,12,Djokja Yogyakarta Indonesia,NaN,NaN
4,13,Triolet Mauritius,-20.057,57.552


In [ ]:
users_data = filtered_user_data.merge(users_df).drop("Unnamed: 0",axis=1)

In [17]:
users_data.head()

,user_id,gender,joinedAt,location,age,lat,lng
0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19,3.590,98.674
1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20,3.590,98.674
2,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47,3.590,98.674
3,1355996271,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,19,NaN,NaN
4,2411726276,female,2012-10-30T05:16:27.136Z,Triolet Mauritius,16,-20.057,57.552


In [18]:
len(users_data)

9922

In [19]:
users_data.isnull().sum()

user_id       0
gender        0
joinedAt      0
location      0
age           0
lat         398
lng         398
dtype: int64

In [20]:
users_data = users_data.dropna()

In [21]:
len(users_data)

9524

In [22]:
users_data.to_csv("users_data.csv")

## Create user onboarding data